In [ ]:
from google.colab import files
 
#uploading d'un fichier CSV que Jamila a crée en concaténant plusieurs fichier csv nettoyé (suppression de duplicats et colonnes inutiles)
uploaded = files.upload()

In [ ]:

import pandas as pd
import requests, json
from pandas.io.json import json_normalize

df= pd.read_csv("bus_mercredi15juin sans 7-8 du 15_06.csv")



In [ ]:
# le fichier est un peu compliqué contient l'heure de passage de chaque bus pour chacun de ses arrêts, toute la journée
df

In [ ]:
#affichage du nombre de bus qui circulent mercredi à Rennes
df["nomcourtligne"].nunique()

In [ ]:
#affichage des noms uniques de toutes les lignes de bus contenu dans ce fichier
df["nomcourtligne"].unique()

In [ ]:
# masque booléen pour ne selectionner que les lignes concernant le bus C1 le bus le plus fréquenté à Rennes 
ligneC1= df[(df["nomcourtligne"]== 'C1')]

# affichage des arrêts du bus C1
ligneC1["nomarret"].unique()


In [ ]:
ligneC1.head()

In [ ]:
ligneC1_arret=ligneC1.groupby(["destination", "idarret", "hours"]).count()[['heure_arrivee']]

ligneC1_arret

In [ ]:
#création d'une fonction qui va me permettre de créer une colonne plage horaire et voir combien de fois le bus passe par plage horaire
def plagehoraire(x):
  if 7<= x <=9:
    return '(1)7-9'
  if 9<= x <= 11:
    return '(2)9-11'
  if 11<= x <= 13:
    return '(3)11-13'
  if 13<= x <= 15:
    return '(4)13-15'
  if 15<= x <= 17:
    return '(5)15-17'
  if 17<= x <= 19:
    return '(6)17-19'
  if 19<= x <= 21:
    return '(7)19-21'
  if 21<= x <= 23:
    return '(8)21-23'
  if 23<= x or x == 0:
    return '(9)23-01'

In [ ]:

#création de la colonne plage horaire
df["tranche_horaire"]= df["hours"].apply(plagehoraire)

# ici je compte le nombre de passage par plage horaire et pour le sens 1 qui est le sens allé 
PassageC1= df[(df["nomcourtligne"]== "C1") & (df["nomarret"]== 'Cesson Hôpital Privé')& (df["sens"]== 1)].groupby(["tranche_horaire","sens"]).count()

In [ ]:
PassageC1= PassageC1.reset_index()

PassageC1

In [ ]:
#la moyenne de passage mercredi
PassageC1['nomcourtligne'].mean()

In [ ]:
# création de la figure de passage du bus par plage horaire 
import plotly.express as px
fig1 = px.bar(PassageC1, x='tranche_horaire', y='nomcourtligne', color ='nomcourtligne', title="Passage du bus C1 par tranche horaire", labels={'nomcourtligne': "Nombre de passages", "tranche_horaire": "Tranche horaire"}, color_continuous_scale = "dense")
fig1.update_layout(barmode='group')
fig1.add_hline(y=8.71, line_dash='dash')
fig1.update_layout(title = {'x':0.5},font = {"family" : "arial"})
fig1.update(layout_yaxis_range = [0,22])
fig1.update_layout(font=dict(size=15,color="black"))
fig1.show()

In [ ]:
# création du fichier html de la figure de passage du bus C1 
fig1.write_html("bus_mercredi.html")